In [123]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.autograd import Variable


In [124]:
im_size = 224

In [165]:
data_dir = 'my_train'
def load_split_train_test(datadir, valid_size = .1):
    train_transforms = transforms.Compose([transforms.Resize([im_size,im_size]),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize([im_size,im_size]),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    print(num_train)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=32)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=32)
    return trainloader, testloader
trainloader, testloader = load_split_train_test(data_dir, .1)
# print(trainloader.dataset.)
print(trainloader.dataset.classes)

2314
['aircrafts', 'birds_', 'cars', 'dogs_', 'flowers_']


In [126]:
import pretrainedmodels

In [183]:
# model = pretrainedmodels.resnet18(num_classes=1000, pretrained='imagenet')
# print(model)
# model.eval()
# # x = None

In [182]:
# # For ResNet18 - caffe
# for param in model.parameters():
#     param.requires_grad = False
    
# # model.fc = nn.Sequential(nn.Linear(2048, 512),
# #                                  nn.ReLU(),
# #                                  nn.Dropout(0.2),
# #                                  nn.Linear(512, 5),
# #                                  nn.LogSoftmax(dim=1))
# model.last_linear = nn.Sequential(nn.Linear(512, 1024),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                   nn.Linear(1024, 1024),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                   nn.Linear(1024, 1024),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                   nn.Linear(1024, 1024),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                   nn.Linear(1024, 1024),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                   nn.Linear(1024, 1024),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                   nn.Linear(1024, 1024),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                   nn.Linear(1024, 1024),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(1024, 2048),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(2048, 512),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(512, 5),
#                                  nn.LogSoftmax(dim=1))
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.last_linear.parameters(), lr=0.003)
# model.to(device)

In [228]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
# model = models.inception_v3(pretrained=True)
# model = models.vgg16(pretrained=True)
# model = models.alexnet(pretrained=True)
# model = models.resnet18(pretrained=True)
# model = models.densenet121(pretrained=True)
model = models.resnet18(pretrained=True)
model.eval()
# print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [135]:
# #for alaexnet
# for param in model.parameters():
#     param.requires_grad = False
    
# model.classifier = nn.Sequential(
#                                 nn.Linear(1024, 4096),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(4096, 4096),
#                                  nn.ReLU(),
#                                  nn.Linear(4096, 5),
#                                  nn.LogSoftmax(dim=1))
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
# model.to(device)

In [31]:
# #for alaexnet
# for param in model.parameters():
#     param.requires_grad = False
    
# model.classifier = nn.Sequential(nn.Dropout(0.2),
#                                 nn.Linear(9216, 4096),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(4096, 4096),
#                                  nn.ReLU(),
#                                  nn.Linear(4096, 5),
#                                  nn.LogSoftmax(dim=1))
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
# model.to(device)

In [32]:
# # for inceptionv3
# for param in model.parameters():
#     param.requires_grad = False

# # model.AuxLogits = None
# # model.fc = nn.Linear(2048, 5)
# model.aux_logits = False
# model.fc = nn.Sequential(nn.Linear(2048, 5),
#                                  nn.LogSoftmax(dim=1))
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
# model.to(device)

In [181]:
# #for VGG16
# for param in model.parameters():
#     param.requires_grad = False
    
# model.classifier = nn.Sequential(nn.Linear(25088, 4096),
#                                  nn.ReLU(),
#                                  nn.Dropout(0.2),
#                                  nn.Linear(4096, 5),
#                                  nn.LogSoftmax(dim=1))
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
# model.to(device)

In [230]:
ct = 0
for child in model.children():
    ct += 1
    if ct <= 6:
        for param in child.parameters():
            param.requires_grad = False


In [231]:
# For ResNet18
# for param in model.parameters():
#     param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(512, 2048),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(2048, 5),
                                 nn.LogSoftmax(dim=1))
# model.fc = nn.Sequential(nn.Linear(512, 5),
#                                  nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [232]:
epochs = 3
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
#         print("shape : ", inputs[0].shape)
#         nlist = []
#         for i in range(64):
#             inputs[i] = inputs[i].view(inputs[i].size(0), -1)
#             nlist.append(p)
#         inputs = torch.FloatTensor(nlist)
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss : {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
torch.save(model, 'aerialmodel.pth')

Epoch 1/3.. Train loss : 1.669.. Test loss: 1.555.. Test accuracy: 0.241
Epoch 1/3.. Train loss : 0.836.. Test loss: 5.015.. Test accuracy: 0.204
Epoch 1/3.. Train loss : 0.144.. Test loss: 0.673.. Test accuracy: 0.914
Epoch 1/3.. Train loss : 0.236.. Test loss: 0.170.. Test accuracy: 0.953
Epoch 1/3.. Train loss : 0.291.. Test loss: 0.402.. Test accuracy: 0.916
Epoch 1/3.. Train loss : 0.165.. Test loss: 0.529.. Test accuracy: 0.934
Epoch 2/3.. Train loss : 0.354.. Test loss: 1.127.. Test accuracy: 0.824
Epoch 2/3.. Train loss : 0.331.. Test loss: 0.623.. Test accuracy: 0.855
Epoch 2/3.. Train loss : 0.170.. Test loss: 0.161.. Test accuracy: 0.941
Epoch 2/3.. Train loss : 0.103.. Test loss: 0.097.. Test accuracy: 0.980
Epoch 2/3.. Train loss : 0.177.. Test loss: 0.056.. Test accuracy: 0.977
Epoch 2/3.. Train loss : 0.175.. Test loss: 0.127.. Test accuracy: 0.955
Epoch 2/3.. Train loss : 0.225.. Test loss: 0.038.. Test accuracy: 0.988
Epoch 3/3.. Train loss : 1.095.. Test loss: 0.867..

In [236]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model=torch.load('aerialmodel.pth')
model=torch.load('best_models/resnet18-97.pth')
# model.eval()

In [237]:
test_transforms = transforms.Compose([transforms.Resize([im_size,im_size]),
                                      transforms.ToTensor()])

def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    inputa = Variable(image_tensor)
    inputa = inputa.to(device)
    output = model(inputa)
    index = output.data.cpu().numpy().argmax()
#     print(index)
    return index

In [238]:
import glob
from PIL import Image

lab = ['aircrafts', 'birds_', 'cars', 'dogs_', 'flowers_']

# trans = transforms.ToPILImage()
# trans1 = transforms.ToTensor()
# test_data_dir = 'coarse_test'
# test_transforms = transforms.Compose([transforms.Resize([224,224]),
#                                       transforms.ToTensor()])
# data = datasets.ImageFolder(test_data_dir, transform=test_transforms)
# classes = data.classes
# loader = torch.utils.data.DataLoader(data)
# dataiter = iter(loader)
# print(dataiter)
# images, labels = dataiter.next()
# print(labels)
# print(len(images))
# fig=plt.figure(figsize=(10,10))
# total_images = len(images)
count = 0
total = 0
for category in os.listdir("coarse_test"):
    for i in os.listdir("coarse_test/" + category):
        image = Image.open("coarse_test/" + category + "/" + i)
#         image = image.resize((299,299), Image.ANTIALIAS)
        index = predict_image(image)
#         sub = fig.add_subplot(1, len(images), ii+1)
        res = category == lab[index]
#         plt.imshow(image)
        print(index)
        if res:
            count+=1
        total+=1
#     break
#         sub.set_title(str(classes[index]) + ":" + str(res))
#         plt.axis('off')
#         plt.imshow(image)

# count = 0
# for ii in range(len(images)):
# #     print(type(images[ii]))
#     trans1 = transforms.ToPILImage()
#     image = trans1(images[ii])
# #     image = to_pil(images[ii])
#     index = predict_image(image)
#     sub = fig.add_subplot(1, len(images), ii+1)
#     res = int(labels[ii]) == index
#     if res:
#         count+=1  
#     sub.set_title(str(classes[index]) + ":" + str(res))
#     plt.axis('off')
#     plt.imshow(image)
# plt.show()
print("Test accuracy : ", count/total)
print(total)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
0
3
3
3
3
3
3
3
3
3
1
3
3
3
3
3
3
3
3
3
3
3
3
Test accuracy :  0.9724770642201835
109


In [217]:
torch.save(model, 'aerialmodel.pth')